In [1]:
# based on Deep Learning Illustrated by Jon Krohn
# https://www.amazon.com/Deep-Learning-Illustrated-Intelligence-Addison-Wesley/dp/0135116694
# in turn based on bit.ly/keonDQN

import os
from collections import deque
import random
import time
import resource
import pickle

import pdb

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model, model_from_json
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

import plotly.express as px
import plotly.graph_objects as go

# requires python 3.6
# conda install -c akode gym
import gym

# set seeds for reproducibility
# np.random.uniform(0,10000) 4465
random.seed(4465)
np.random.seed(4465)
tf.random.set_seed(4465)


In [2]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.98
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001
        self.model = self.build_model()
        self.memory = pd.DataFrame(columns=["state", "action", "next_state", "reward", "done"])
        self.memory_size=200000
        self.results = []
        self.train_batch_size=8
        self.timestep=0
        self.summary_interval=10
        
    def build_model(self,
                    n_hidden_layers=2, 
                    hidden_layer_size=16, 
                    activation='relu',
                    reg_penalty=0.001,
                    dropout=0.0675,
                    verbose=True
                   ):
        """return keras NN model per inputs
        input is a state - array of size state_size
        output is an array of action values - array of size action_size
        """

        model = Sequential()

        for i in range(n_hidden_layers):
            if verbose:
                print("layer %d size %d, %s, reg_penalty %.8f, dropout %.3f" % (i + 1, 
                                                                                hidden_layer_size, 
                                                                                activation,
                                                                                reg_penalty,
                                                                                dropout,
                                                                               ))
            # add dropout, but not on inputs, only between hidden layers
            if i and dropout:
                model.add(Dropout(dropout))

            if i==0: # first layer, specify input shape
                model.add(Dense(input_shape=(state_size,),
                                units = hidden_layer_size, 
                                activation = activation,
                                kernel_initializer = keras.initializers.glorot_uniform(),
                                kernel_regularizer=keras.regularizers.l2(reg_penalty),
                                name = "Dense%02d" % i))
            else: #use implicit input shape
                model.add(Dense(units = hidden_layer_size, 
                                activation = activation,
                                kernel_initializer = keras.initializers.glorot_uniform(),
                                kernel_regularizer=keras.regularizers.l2(reg_penalty),
                                name = "Dense%02d" % i))

        model.add(Dense(self.action_size, activation='linear'))

        if verbose:
            print(model.summary())

        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))

        return model
        
    def remember(self, state, action, reward, next_state, done):
        # append in place
        self.memory.loc[self.memory.shape[0]]=[state[0], action, next_state[0], reward, done]
            
    def train(self, sample_size, start_epoch=0):
        # truncate memory
        self.memory = self.memory[-self.memory_size:]
        # sample sample_size observations from memory
        minibatch = self.memory.sample(n=sample_size)
        
        # target is our best estimate of value of each action
        X_fit = np.concatenate(minibatch['state'].values)
        X_fit = X_fit.reshape((sample_size, self.state_size))
        Y_pred = self.model.predict(X_fit)

        # we don't just fit model against model's own prediction, that would get us nowhere
        # we improve the target by what we learned about the action we actually took
        # value is reward obtained + predicted value of the observed next state
        minibatch['target_observed'] = minibatch['reward']
        # if done, target is the reward 
        # reward by gym env is only 1 for each timestep of survival
        # but we also added a reward of -10 on failure
        # if not done, add gamma discount rate * Q-value prediction for the observed next state
        not_done = minibatch.loc[minibatch['done'] == False]
        X_observed = np.concatenate(not_done['next_state'].values)
        X_observed = X_observed.reshape((not_done.shape[0], self.state_size))
        # run all predictions at once
        # iterates faster but does not train after each prediction
        y_observed_pred = np.amax(self.model.predict(X_observed), axis=1)
        minibatch.loc[minibatch['done'] == False, 'target_observed'] += self.gamma * y_observed_pred
        # vectorized vlookup - update y_pred column specified by action using target_observed
        np.put_along_axis(Y_pred, 
                          minibatch['action'].astype(int).values.reshape(sample_size,1), 
                          minibatch['target_observed'].values.reshape(sample_size,1),
                          axis=1)
        # fit model against improved target
        # arbitrary 8 batch size to reduce variance a little and speed up fit
        self.model.fit(X_fit, Y_pred, 
                       epochs=1, initial_epoch=start_epoch,
                       batch_size=self.train_batch_size, 
                       verbose=0)
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def reset(self):
        self.timestep = 0
    
    def increment_time(self):
        self.timestep +=1

    def score_episode(self, e, n_episodes):
        self.save_score()
        avglen=min(len(self.results), self.summary_interval)
        print("{} episode: {}/{}, score: {}, {}-episode avg: {:.1f} epsilon: {:.02}"
              .format(time.strftime("%H:%M:%S"), e, n_episodes, self.timestep, 
                      avglen, sum(self.results[-avglen:])/avglen, self.epsilon))
        
    def save_score(self):
        self.results.append(self.timestep)    
    
    def load(self, filename, memory=True):
        self.model = load_model("%s.h5" % filename)
        pickledict = pickle.load(open( "%s.p" % filename, "rb"))
        self.memory = pickledict['memory']
        self.results = pickledict['results']
        self.epsilon = pickledict['epsilon']
        print("loaded %d results, %d rows of memory, epsilon %.4f" % (len(self.results),
                                                                      len(self.memory),
                                                                      self.epsilon))

    def save(self, pathname, memory=True):
        fullname = "%s%04d" % (pathname, len(self.results))
        self.model.save("%s.h5" % fullname)
        pickledict = {
            'memory': self.memory,
            'results': self.results,
            'epsilon': self.epsilon,
        }
        pickle.dump( pickledict, open( "%s.p" % fullname, "wb" ) )
        print("saved model to %s" % fullname)

        

In [3]:
env = gym.make('CartPole-v1')


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/Users/druce/anaconda3/envs/python36/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning:

Parameters to load are deprecated.  Call .resolve and .require separately.



In [8]:
env.action_space.n

2

In [3]:
#https://gym.openai.com/envs/CartPole-v0/
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
sample_size = 128
n_episodes = 400
fail_penalty = -20

output_dir = 'model_output/cartpole/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/Users/druce/anaconda3/envs/python36/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning:

Parameters to load are deprecated.  Call .resolve and .require separately.



In [4]:
agent = DQNAgent(state_size, action_size)

layer 1 size 16, relu, reg_penalty 0.00100000, dropout 0.068
layer 2 size 16, relu, reg_penalty 0.00100000, dropout 0.068
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense00 (Dense)              (None, 16)                80        
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
Dense01 (Dense)              (None, 16)                272       
_________________________________________________________________
dense (Dense)                (None, 2)                 34        
Total params: 386
Trainable params: 386
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
# load earlier model
# start_epoch=1000
# loadmodel = '%04d' % start_epoch
# agent.load(output_dir + 'model_' + loadmodel)
# n_episodes = 50
# agent.epsilon = 0.094


In [6]:
for e in range(n_episodes):
    print ('Memory usage: %s (kb)' % resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)
    
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    agent.reset()
    done = False
    
    while not done:
        env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        if done and e < (n_episodes-1):
            reward = fail_penalty
        next_state = next_state.reshape([1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            agent.score_episode(e, n_episodes)
        else:
            agent.increment_time()
    
    if len(agent.memory) > sample_size*2:
        agent.train(max(sample_size, int(agent.memory.shape[0] *0.05)))
        
    if e and (e+1) % agent.summary_interval == 0:
        agent.save(output_dir + "model_")


Memory usage: 257581056 (kb)
17:18:16 episode: 0/400, score: 14, 1-episode avg: 14.0 epsilon: 1.0
Memory usage: 260386816 (kb)
17:18:17 episode: 1/400, score: 9, 2-episode avg: 11.5 epsilon: 1.0
Memory usage: 260386816 (kb)
17:18:17 episode: 2/400, score: 11, 3-episode avg: 11.3 epsilon: 1.0
Memory usage: 260386816 (kb)
17:18:17 episode: 3/400, score: 26, 4-episode avg: 15.0 epsilon: 1.0
Memory usage: 260386816 (kb)
17:18:17 episode: 4/400, score: 7, 5-episode avg: 13.4 epsilon: 1.0
Memory usage: 260386816 (kb)
17:18:18 episode: 5/400, score: 68, 6-episode avg: 22.5 epsilon: 1.0
Memory usage: 260591616 (kb)
17:18:19 episode: 6/400, score: 14, 7-episode avg: 21.3 epsilon: 1.0
Memory usage: 260616192 (kb)
17:18:19 episode: 7/400, score: 23, 8-episode avg: 21.5 epsilon: 1.0
Memory usage: 260657152 (kb)
17:18:19 episode: 8/400, score: 11, 9-episode avg: 20.3 epsilon: 1.0
Memory usage: 260685824 (kb)
17:18:20 episode: 9/400, score: 31, 10-episode avg: 21.4 epsilon: 1.0
saved model to model_

KeyboardInterrupt: 

In [7]:
df = pd.DataFrame({'timesteps': agent.results})
df['avg'] = df['timesteps'].rolling(10).mean() 
df


loaded 1000 results, 199698 rows of memory, epsilon 0.0100


,timesteps,avg
0,14,NaN
1,10,NaN
2,12,NaN
3,20,NaN
4,10,NaN
...,...,...
995,117,230.8
996,114,192.3
997,113,153.7
998,499,192.2


In [13]:
# chart timesteps vs. episodes
def rlplot(agent):
    df = pd.DataFrame({'timesteps': agent.results})
    df['avg'] = df['timesteps'].rolling(10).mean() 

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['timesteps'],
                             mode='markers',
                             name='timesteps',
                             marker=dict(
                                 color='LightSkyBlue',
                                 size=5,
                             ),
                            ))

    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['avg'],
                             mode='lines',
                             name='moving average'))

    fig.update_layout(
        title= dict(text='Cartpole DQN Agent Training Progress',
                    x=0.5,
                    xanchor='center'),
        xaxis=dict(
            title="Episodes",
            linecolor='black',
            linewidth=1,
            mirror=True
        ),
        yaxis=dict(
            title="Completed Timesteps",
            linecolor='black',
            linewidth=1,
            mirror=True
        ),
        legend=go.layout.Legend(
            x=0.01,
            y=0.99,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=12,
                color="black"
            ),
            #bgcolor="LightSteelBlue",
            bordercolor="Black",
            borderwidth=1,
        ),
    )

    return fig.show()

start_epoch=400
loadmodel = '%04d' % start_epoch
agent.load(output_dir + 'model_' + loadmodel)
rlplot(agent)



loaded 400 results, 39436 rows of memory, epsilon 0.1423


In [12]:
start_epoch=1000
loadmodel = '%04d' % start_epoch
agent.load(output_dir + 'model_' + loadmodel)
rlplot(agent)

loaded 1000 results, 199698 rows of memory, epsilon 0.0100


In [7]:
# training as above does well up to a point but not very stable
# sometimes performance goes off a cliff esp with more complex NNs like 2x32
# continuing to train sometimes results in forgetting what it learned
# also on my machine tensorflow leaks memory, can't train long without restarting
# trained repeatedly, when it fell off a cliff restarted using best previous model
# early stopping after achieving a very good model
# saved best model, run it here without epsilon random exploration, or training

agent.load('good_new')
agent.epsilon = 0.01
print(agent.model.summary())

for e in range(10):
    print ('Memory usage: %s (kb)' % resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)
    
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    agent.reset()
    done = False
    
    while not done:
        env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else fail_penalty
        next_state = next_state.reshape([1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            agent.score_episode(e, n_episodes)
        else:
            agent.increment_time()
            
    # don't train or save after each episode
    


loaded 0 results, 1132 rows of memory, epsilon 0.0100
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense00 (Dense)              (None, 32)                160       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
Dense01 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense (Dense)                (None, 2)                 66        
Total params: 1,282
Trainable params: 1,282
Non-trainable params: 0
_________________________________________________________________
None
Memory usage: 320581632 (kb)
17:19:25 episode: 0/400, score: 499, 1-episode avg: 499.0 epsilon: 0.01
Memory usage: 611155968 (kb)


KeyboardInterrupt: 